In [1]:
import pandas as pd
import os
import glob
import numpy as np

In [2]:
# Config
BS_CLEAN_DIR = '../balance_sheet_clean_label'
IS_CLEAN_DIR = '../income_statement_clean_label'
BS_MAPPING = 'bs_calculated_operating_mapping.csv'
IS_MAPPING = 'is_calculated_operating_expense_mapping.csv'

In [3]:
# --- Sort and Save Existing CSV ---
# Parameters
SORT_COLUMN = 'avg_order_fwd'
# SORT_COLUMN = 'avg_order_rev'
# SORT_COLUMN = 'frequency'
ASCENDING = True            # Set to False for descending order
FILE_PATH = IS_MAPPING     # Uses the variable from the config cell

try:
    if os.path.exists(FILE_PATH):
        df_mapping = pd.read_csv(FILE_PATH)
        
        # Check if column exists
        if SORT_COLUMN in df_mapping.columns:
            # Sort
            df_mapping = df_mapping.sort_values(by=SORT_COLUMN, ascending=ASCENDING)
            
            # Save
            df_mapping.to_csv(FILE_PATH, index=False)
            print(f"Successfully sorted '{FILE_PATH}' by '{SORT_COLUMN}'")
            print(df_mapping.head())
        else:
            print(f"Error: Column '{SORT_COLUMN}' not found in {FILE_PATH}")
            print(f"Available columns: {list(df_mapping.columns)}")
    else:
        print(f"Error: File '{FILE_PATH}' does not exist.")
except Exception as e:
    print(f"An error occurred: {e}")

Successfully sorted 'is_calculated_operating_expense_mapping.csv' by 'avg_order_fwd'
     standardized_name  frequency  avg_order_fwd  avg_order_rev  \
0              revenue         29           1.00          13.72   
1        gross_revenue          2           2.00          19.00   
2      segment_revenue         78           2.46          17.51   
3  revenue_adjustments          2           3.00          18.00   
4      sales_discounts          2           4.00          17.00   

   is_calculated  is_operating is_expense  
0          False          True      False  
1          False          True      False  
2          False          True      False  
3          False          True       True  
4          False          True       True  


In [4]:
# Parameters
OUTPUT_FILE = IS_MAPPING     # Use BS_MAPPING or IS_MAPPING based on what you are processing
CLEAN_DIR = IS_CLEAN_DIR     # Use BS_CLEAN_DIR or IS_CLEAN_DIR
# 1. Load Existing Mapping (to preserve manual work)
if os.path.exists(OUTPUT_FILE):
    existing_df = pd.read_csv(OUTPUT_FILE)
    print(f"Loaded existing mapping with {len(existing_df)} rows.")
else:
    existing_df = pd.DataFrame()
    print("No existing mapping found. Creating new.")
# 2. Recalculate Stats from Clean Data
files = glob.glob(os.path.join(CLEAN_DIR, "*.csv"))
df_list = []
print(f"Reading {len(files)} files from {CLEAN_DIR}...")
for f in files:
    try:
        d = pd.read_csv(f)
        df_list.append(d)
    except Exception as e:
        print(f"Error reading {f}: {e}")
if not df_list:
    raise ValueError("No CSV files found.")
combined_df = pd.concat(df_list, ignore_index=True)
# Calculate Orders
combined_df['fwd_order'] = combined_df.groupby('company').cumcount() + 1
combined_df['co_len'] = combined_df.groupby('company')['standardized_name'].transform('count')
combined_df['rev_order'] = combined_df['co_len'] - combined_df['fwd_order'] + 1
# Clean is_calculated
if 'is_calculated' in combined_df.columns:
    combined_df['is_calculated'] = combined_df['is_calculated'].replace(
        {'True': True, 'False': False, 'TRUE': True, 'FALSE': False}
    )
def get_consistent_calculated(series):
    uniques = series.dropna().unique()
    if len(uniques) == 1:
        return uniques[0]
    return np.nan
# Aggregate
new_stats = combined_df.groupby('standardized_name').agg(
    frequency=('standardized_name', 'count'),
    avg_order_fwd=('fwd_order', 'mean'),
    avg_order_rev=('rev_order', 'mean'),
    new_is_calculated=('is_calculated', get_consistent_calculated)
).reset_index()
# Rounding
new_stats['avg_order_fwd'] = new_stats['avg_order_fwd'].round(2)
new_stats['avg_order_rev'] = new_stats['avg_order_rev'].round(2)
# 3. Merge and Update
# We want to keep all rows from new_stats (in case there are new standardized names)
# We want to preserve manual columns from existing_df if they exist
if not existing_df.empty:
    # Merge on standardized_name
    merged = pd.merge(new_stats, existing_df, on='standardized_name', how='left', suffixes=('', '_old'))
    
    # Update is_calculated logic:
    # If the user manually edited it, we might want to keep it? 
    # Your request says "fill ... with content in the csv", implying we overwrite Calculated 
    # but keep Operating.
    # Let's overwrite is_calculated with the newly derived one (since it comes from the clean labels)
    merged['is_calculated'] = merged['new_is_calculated']
    
    # Preserve is_operating (and is_expense if it exists)
    if 'is_operating' not in merged.columns:
        merged['is_operating'] = np.nan
    
    # If specifically processing Income Statement, ensure is_expense exists
    if 'is_expense' not in merged.columns and 'is_expense' in existing_df.columns:
         merged['is_expense'] = existing_df['is_expense'] 
    elif 'is_expense' not in merged.columns:
         merged['is_expense'] = np.nan
    # Select and Clean Columns
    cols = ['standardized_name', 'frequency', 'avg_order_fwd', 'avg_order_rev', 'is_calculated', 'is_operating']
    if 'is_expense' in merged.columns:
        cols.append('is_expense')
        
    final_df = merged[cols]
else:
    # First run
    final_df = new_stats.rename(columns={'new_is_calculated': 'is_calculated'})
    final_df['is_operating'] = np.nan

# Check if standardized_name column exists and filter
if 'standardized_name' in final_df.columns:
    final_df = final_df[~final_df['standardized_name'].str.endswith('_header')]

# 4. Save
# Sort by frequency or whatever your preference is before saving
final_df = final_df.sort_values(by=['avg_order_fwd'])
final_df.to_csv(OUTPUT_FILE, index=False)
print(f"Success! Updated {OUTPUT_FILE} with {len(final_df)} rows.")
print(final_df.head())

Loaded existing mapping with 54 rows.
Reading 60 files from ../income_statement_clean_label...
Success! Updated is_calculated_operating_expense_mapping.csv with 54 rows.
      standardized_name  frequency  avg_order_fwd  avg_order_rev  \
41              revenue         29           1.00          13.72   
15        gross_revenue          2           2.00          19.00   
51      segment_revenue         78           2.46          17.51   
42  revenue_adjustments          2           3.00          18.00   
45      sales_discounts          2           4.00          17.00   

    is_calculated is_operating is_expense  
41          False         True      False  
15          False         True      False  
51          False         True      False  
42          False         True       True  
45          False         True       True  


In [5]:
# Parameters
OUTPUT_FILE = BS_MAPPING     # Use BS_MAPPING or IS_MAPPING based on what you are processing
CLEAN_DIR = BS_CLEAN_DIR     # Use BS_CLEAN_DIR or IS_CLEAN_DIR
# 1. Load Existing Mapping (to preserve manual work)
if os.path.exists(OUTPUT_FILE):
    existing_df = pd.read_csv(OUTPUT_FILE)
    print(f"Loaded existing mapping with {len(existing_df)} rows.")
else:
    existing_df = pd.DataFrame()
    print("No existing mapping found. Creating new.")
# 2. Recalculate Stats from Clean Data
files = glob.glob(os.path.join(CLEAN_DIR, "*.csv"))
df_list = []
print(f"Reading {len(files)} files from {CLEAN_DIR}...")
for f in files:
    try:
        d = pd.read_csv(f)
        df_list.append(d)
    except Exception as e:
        print(f"Error reading {f}: {e}")
if not df_list:
    raise ValueError("No CSV files found.")
combined_df = pd.concat(df_list, ignore_index=True)
# Calculate Orders
combined_df['fwd_order'] = combined_df.groupby('company').cumcount() + 1
combined_df['co_len'] = combined_df.groupby('company')['standardized_name'].transform('count')
combined_df['rev_order'] = combined_df['co_len'] - combined_df['fwd_order'] + 1
# Clean is_calculated
if 'is_calculated' in combined_df.columns:
    combined_df['is_calculated'] = combined_df['is_calculated'].replace(
        {'True': True, 'False': False, 'TRUE': True, 'FALSE': False}
    )
def get_consistent_calculated(series):
    uniques = series.dropna().unique()
    if len(uniques) == 1:
        return uniques[0]
    return np.nan
# Aggregate
new_stats = combined_df.groupby('standardized_name').agg(
    frequency=('standardized_name', 'count'),
    avg_order_fwd=('fwd_order', 'mean'),
    avg_order_rev=('rev_order', 'mean'),
    new_is_calculated=('is_calculated', get_consistent_calculated)
).reset_index()
# Rounding
new_stats['avg_order_fwd'] = new_stats['avg_order_fwd'].round(2)
new_stats['avg_order_rev'] = new_stats['avg_order_rev'].round(2)
# 3. Merge and Update
# We want to keep all rows from new_stats (in case there are new standardized names)
# We want to preserve manual columns from existing_df if they exist
if not existing_df.empty:
    # Merge on standardized_name
    merged = pd.merge(new_stats, existing_df, on='standardized_name', how='left', suffixes=('', '_old'))
    
    # Update is_calculated logic:
    # If the user manually edited it, we might want to keep it? 
    # Your request says "fill ... with content in the csv", implying we overwrite Calculated 
    # but keep Operating.
    # Let's overwrite is_calculated with the newly derived one (since it comes from the clean labels)
    merged['is_calculated'] = merged['new_is_calculated']
    
    # Preserve is_operating (and is_expense if it exists)
    if 'is_operating' not in merged.columns:
        merged['is_operating'] = np.nan
    
    # If specifically processing Balance Sheet, ensure nonoperating_category exists
    if 'nonoperating_category' not in merged.columns and 'nonoperating_category' in existing_df.columns:
         merged['nonoperating_category'] = existing_df['nonoperating_category'] 
    elif 'nonoperating_category' not in merged.columns:
         merged['nonoperating_category'] = np.nan
    # Select and Clean Columns
    cols = ['standardized_name', 'frequency', 'avg_order_fwd', 'avg_order_rev', 'is_calculated', 'is_operating']
    if 'nonoperating_category' in merged.columns:
        cols.append('nonoperating_category')
        
    final_df = merged[cols]
else:
    # First run
    final_df = new_stats.rename(columns={'new_is_calculated': 'is_calculated'})
    final_df['nonoperating_category'] = np.nan
    
# Check if standardized_name column exists and filter
if 'standardized_name' in final_df.columns:
    final_df = final_df[~final_df['standardized_name'].str.endswith('_header')]

# 4. Save
# Sort by frequency or whatever your preference is before saving
final_df = final_df.sort_values(by=['avg_order_fwd'])
final_df.to_csv(OUTPUT_FILE, index=False)
print(f"Success! Updated {OUTPUT_FILE} with {len(final_df)} rows.")
print(final_df.head())

Loaded existing mapping with 104 rows.
Reading 53 files from ../balance_sheet_clean_label...
Success! Updated bs_calculated_operating_mapping.csv with 104 rows.
             standardized_name  frequency  avg_order_fwd  avg_order_rev  \
13        cash_and_equivalents         53           1.32          29.91   
88             restricted_cash          8           2.38          36.75   
95      short_term_investments         37           2.70          30.95   
1          accounts_receivable         44           3.45          28.73   
99  total_cash_and_investments          4           3.50          29.25   

    is_calculated is_operating   nonoperating_category  
13          False        False                    cash  
88          False        False                    cash  
95          False        False  short_term_investments  
1           False         True                     NaN  
99           True        False                     NaN  
